## BERT without registers

In [13]:
from transformers import AutoTokenizer
from model import RegBertForQA  
from ExplanationGenerator import Generator 
from captum.attr import visualization
from BERT import BertModel
import torch

model = RegBertForQA.from_pretrained("fine_tuned_model_orig", num_registers = 0).to("cuda")
model.eval()

tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model_orig")
explanations = Generator(model)

question, context = "What is a way to increase your wound healing speed?", \
"Wound care encourages and speeds wound healing via cleaning and protection from reinjury or infection. Depending on each patient's needs, it can range from the simplest first aid to entire nursing specialties such as wound, ostomy, and continence nursing and burn center care."

encoding = tokenizer(
    question,
    context,
    return_tensors="pt",
)
input_ids = encoding["input_ids"].to("cuda")
attention_mask = encoding["attention_mask"].to("cuda")

expl = explanations.generate_LRP(
    input_ids=input_ids, 
    attention_mask=attention_mask, 
    start_layer=0
)[0]

from regbertfor QA, num_reg= 0


Some weights of RegBert were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.embeddings.position_ids', 'bert.reg_pos', 'bert.reg_tokens']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RegBertForQA were not initialized from the model checkpoint at fine_tuned_model_orig and are newly initialized: ['bert.embeddings.position_ids', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'bert.reg_pos', 'bert.reg_tokens']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


outputs: torch.Size([1, 71, 768])
Start logits shape: torch.Size([1, 71])
End logits shape: torch.Size([1, 71])


In [15]:
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

outputs = model(
    input_ids=input_ids, attention_mask=attention_mask
)
num_registers = model.bert.num_registers
start_logits = outputs.start_logits
end_logits = outputs.end_logits

start_probs = torch.softmax(start_logits, dim=-1)
end_probs = torch.softmax(end_logits, dim=-1)

start_index = torch.argmax(start_probs, dim=-1).item()
end_index = torch.argmax(end_probs, dim=-1).item()

adjusted_start_index = max(0, start_index - num_registers)
adjusted_end_index = max(0, end_index - num_registers)

pred_prob = ((start_probs[0, start_index] + end_probs[0, end_index]) / 2).item()

answer_ids = input_ids[0, adjusted_start_index: adjusted_end_index + 1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
predicted_answer = tokenizer.convert_tokens_to_string(answer_tokens)
all_tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())

vis_data_record = visualization.VisualizationDataRecord(
    word_attributions=expl,
    pred_prob=pred_prob,
    pred_class=predicted_answer,
    true_class="",  
    attr_class="",
    attr_score=1,
    raw_input_ids=all_tokens,
    convergence_score=1
)
print(question)
print(answer)
# Visualize attributions
visualization.visualize_text([vis_data_record])


What is a way to increase your wound healing speed?
[{'answer_start': 129, 'text': 'discourse between two or more people holding different points of view about a subject but wishing to establish the truth through reasoned arguments'}]


## BERT With registers

In [11]:
from transformers import AutoTokenizer
from model import RegBertForQA  
from ExplanationGenerator import Generator 
from captum.attr import visualization
from BERT import BertModel
import torch

model = RegBertForQA.from_pretrained("Archive/model_num_reg_50").to("cuda")
model.eval()

tokenizer = AutoTokenizer.from_pretrained("Archive/tokenizer_num_reg_50")
explanations = Generator(model)

question, context = "What is a way to increase your wound healing speed?", \
"Wound care encourages and speeds wound healing via cleaning and protection from reinjury or infection. Depending on each patient's needs, it can range from the simplest first aid to entire nursing specialties such as wound, ostomy, and continence nursing and burn center care."

encoding = tokenizer(
    question,
    context,
    return_tensors="pt",
)
input_ids = encoding["input_ids"].to("cuda")
attention_mask = encoding["attention_mask"].to("cuda")

expl = explanations.generate_LRP(
    input_ids=input_ids, 
    attention_mask=attention_mask,  
    start_layer=0
)[0]

from regbertfor QA, num_reg= 50


Some weights of RegBert were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.embeddings.position_ids', 'bert.reg_pos', 'bert.reg_tokens']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


outputs: torch.Size([1, 121, 768])
Start logits shape: torch.Size([1, 71])
End logits shape: torch.Size([1, 71])


In [12]:
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())
outputs = model(
    input_ids=input_ids, attention_mask=attention_mask
)
num_registers = model.bert.num_registers
start_logits = outputs.start_logits
end_logits = outputs.end_logits

start_probs = torch.softmax(start_logits, dim=-1)
end_probs = torch.softmax(end_logits, dim=-1)

start_index = torch.argmax(start_probs, dim=-1).item()
end_index = torch.argmax(end_probs, dim=-1).item()

adjusted_start_index = max(0, start_index)
adjusted_end_index = max(0, end_index)

pred_prob = ((start_probs[0, start_index] + end_probs[0, end_index]) / 2).item()

answer_ids = input_ids[0, adjusted_start_index: adjusted_end_index + 1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
predicted_answer = tokenizer.convert_tokens_to_string(answer_tokens)
all_tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
reg_tokens = [f'[REG{i}]' for i in range(num_registers)]
reg_tokens.extend(all_tokens)
vis_data_record = visualization.VisualizationDataRecord(
    word_attributions=expl,
    pred_prob=pred_prob,
    pred_class=predicted_answer,
    true_class="",  
    attr_class="",
    attr_score=1,
    raw_input_ids=reg_tokens,
    convergence_score=1
)
print(adjusted_start_index, adjusted_end_index)
print(question)
# Visualize attributions
visualization.visualize_text([vis_data_record])


13 29
What is a way to increase your wound healing speed?
